In [1]:
import mat73
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from IPython.display import display
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd 

In [2]:
def convert_mat_np(gender=None):
    f_file = "../raw_data/Emotrans1_girl_data_preprocessed_42.mat"
    m_file = "../raw_data/Emotrans1_Boy_data_preprocessed_42.mat"
    # m_file = "../raw_data/Emotrans1_Boy_data_preprocessed_33.mat"
    # f_file = "../raw_data/Emotrans1_girl_data_preprocessed_33.mat"
    if gender == "f":
        data_dict_female = mat73.loadmat(f_file, use_attrdict=True)
        return np.array(data_dict_female["All_Feature"])
    elif gender == 'm':
        data_dict_female = mat73.loadmat(m_file, use_attrdict=True)
        return np.array(data_dict_female["All_Feature"])  
    else: 
        raise Exception("gender not defined")

In [3]:
m = convert_mat_np("m")
f = convert_mat_np("f")
combined = np.concatenate((m,f),axis=0)

<ipython-input-2-fe3f77dfe444>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(data_dict_female["All_Feature"])
<ipython-input-2-fe3f77dfe444>:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(data_dict_female["All_Feature"])


In [4]:
combined.shape

(42, 4, 4)

In [5]:
# type: 0 --> average 3+4 and 1+2; 1--> use only 3 and 2  
def get_pos_or_neg(data,position,type=0):
    arr = []
    index = 0
    for i in range(data.shape[0]):
#         print(data[i].shape)
        if type == 0:
            diff = (data[i][position[0][0]][position[0][1]]+data[i][position[1][0]][position[1][1]])/2 - (data[i][position[3][0]][position[3][1]]+data[i][position[2][0]][position[2][1]])/2
        elif type == 1:
             diff = (data[i][position[1][0]][position[1][1]] - data[i][position[2][0]][position[2][1]])/2
        arr.append(diff)
    return np.array(arr)

In [6]:
# select frequenceis and (0-4s -> 0 or 0.5-4.5s -> 1)
# output: x 
def choose_freq(data=None,freq = None, sec=None):
    if freq:
        data = np.delete(data,freq,axis=3)
    if sec == 0:
        for i in range(data.shape[0]):
            data[i] = np.delete(data[i], 1, axis=3)
        return data
    elif sec == 1:
        for i in range(data.shape[0]):
            data[i] = np.delete(data[i], 0, axis=3) 
        return data
    else:
        print('sec not specified')
        return data 

In [7]:
# reshape to 128*8*2 = 2048
# return x 
def squeeze_feature_size(data):
    new_data = []
    for d in range(data.shape[0]):
#         for i in range(df[d].shape[0]):
        size = data[d].shape
        new_data.append(data[d].reshape(size[0], size[1]*size[2]*size[3]))  
    return np.array(new_data)

In [8]:
# generate labels for x 
# input: data and label_type (0:zeros, 1: ones )
def generate_labels(df,type=None):
    arr = []
    label = []
    label_type = -1
    if type == 0:
        label_type = 0
    elif type == 1:
        label_type = 1
    else:
        print("Wrong type - get_pos_or_neg()")
        return 
    for d in range(df.shape[0]):
#         for i in range(df[d].shape[0]):
        size = df[d].shape
#         print(size)
        if label_type == 1:
            label.append(np.ones((size[0],1)))
        else:
            label.append(np.zeros((size[0],1)))
    return np.array(label)

In [9]:
# combine neg and postive for boys and girls separately
def combine_net_neg(netX,netY,negX,negY):
    new_x, new_y = [],[]
    for i in range(netX.shape[0]):
        new_x.append(np.concatenate((netX[i],negX[i]),axis=0))
        new_y.append(np.concatenate((netY[i],negY[i]),axis=0))
    return np.array(new_x),np.array(new_y)

In [10]:
# get x and y training set for leave one out 
def get_x_y_train(x,y):
    new_x = x[0]
    new_y = y[0]
    for i in range(1,len(x)):
        new_x=np.append(new_x,x[i],axis=0)
        new_y=np.append(new_y,y[i],axis=0)
    return new_x,new_y

In [11]:
# define model 
model = RandomForestClassifier(n_estimators=150) 

In [12]:
pos = [[0,3],[0,2],[0,1],[0,0]]
neg = [[1,3],[1,2],[1,1],[1,0]]
pos_df_m = get_pos_or_neg(m,pos,1)
neg_df_m = get_pos_or_neg(m,neg,1)
pos_df_f = get_pos_or_neg(f,pos,1)
neg_df_f = get_pos_or_neg(f,neg,1)


<ipython-input-5-b80872df8784>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(arr)


In [14]:
def get_new_features(df):
    new_df=[]
    for i in range(df.shape[0]):
        temp = df[i][:,:,1:3,:]
        temp2 = df[i][:,:,4:6,:]
        new_df.append(np.concatenate((temp,temp2),axis=2))
    return np.array(new_df)

In [15]:
def model_train(x_train,x_test,y_train,y_test,model):
    model.fit(x_train,y_train.ravel())
    y_pred = model.predict(x_test)
    return y_test, y_pred

In [16]:
# get x train and test set for witin CV 
def get_x_train_test(data,index):
    x_test = data[index]
    x_train = np.delete(data,index,axis=0)
    return x_train,x_test.reshape(1,-1)

# get y train and test set for witin CV 
def get_y_train_test(data,index):
    y_test = data[index]
    y_train = np.delete(data,index,axis=0)
    return y_train.astype(int).ravel(),y_test.astype(int)

In [17]:
pos_df_m = get_new_features(pos_df_m)
neg_df_m = get_new_features(neg_df_m)
pos_df_f = get_new_features(pos_df_f)
neg_df_f = get_new_features(neg_df_f)

<ipython-input-14-be03b35ddd19>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(new_df)


In [18]:
pos_sequeezed_m = squeeze_feature_size(pos_df_m)
neg_sequeezed_m = squeeze_feature_size(neg_df_m)
pos_sequeezed_f = squeeze_feature_size(pos_df_f)
neg_sequeezed_f = squeeze_feature_size(neg_df_f)

<ipython-input-7-f7e19f44a14e>:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(new_data)


In [19]:
pos_labels_m = generate_labels(pos_sequeezed_m,1)
neg_labels_m = generate_labels(neg_sequeezed_m,0)
pos_labels_f = generate_labels(pos_sequeezed_f,1)
neg_labels_f = generate_labels(neg_sequeezed_f,0)

<ipython-input-8-6902f150008f>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(label)


In [20]:
boy_x,boy_y = combine_net_neg(pos_sequeezed_m,pos_labels_m,neg_sequeezed_m,neg_labels_m)
girl_x,girl_y = combine_net_neg(pos_sequeezed_f,pos_labels_f,neg_sequeezed_f,neg_labels_f)

<ipython-input-9-ef61bbcee622>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(new_x),np.array(new_y)


In [21]:
# combine boys and girls 
x = np.concatenate((boy_x,girl_x),axis=0)
y = np.concatenate((boy_y,girl_y),axis=0)

In [72]:
all_accuracy=[]
for d in range(x.shape[0]):
    print("leave one out # person:",d)
    if d == 0:
        x_train,y_train = get_x_y_train(x[d+1:],y[d+1:])
        x_test,y_test =x[d],y[d]
    elif d == x.shape[0]-1:
        x_train,y_train = get_x_y_train(x[:d],y[:d])
        x_test,y_test = x[d],y[d]
    else:
        x_train,y_train = get_x_y_train(np.append(x[:d],x[d+1:],axis=0),np.append(y[:d],y[d+1:],axis=0))
        x_test,y_test = x[d],y[d]
    # print(x_train.shape,x_test.shape)
    # print(y_train.shape,y_test.shape)
    # return
    y_test,y_pred=model_train(x_train,x_test,y_train,y_test,model)
    accuracy = accuracy_score(y_test,y_pred)
    all_accuracy.append(accuracy)
print(all_accuracy)

leave one out # person: 0
leave one out # person: 1
leave one out # person: 2
leave one out # person: 3
leave one out # person: 4
leave one out # person: 5
leave one out # person: 6
leave one out # person: 7
leave one out # person: 8
leave one out # person: 9
leave one out # person: 10
leave one out # person: 11
leave one out # person: 12
leave one out # person: 13
leave one out # person: 14
leave one out # person: 15
leave one out # person: 16
leave one out # person: 17
leave one out # person: 18
leave one out # person: 19
leave one out # person: 20
leave one out # person: 21
leave one out # person: 22
leave one out # person: 23
leave one out # person: 24
leave one out # person: 25
leave one out # person: 26
leave one out # person: 27
leave one out # person: 28
leave one out # person: 29
leave one out # person: 30
leave one out # person: 31
leave one out # person: 32
leave one out # person: 33
leave one out # person: 34
leave one out # person: 35
leave one out # person: 36
leave one o

In [73]:
for _ in range(10):
    all_accuracy.append(-1)
df = pd.read_csv('feb_27.csv')
print(len(all_accuracy))
df["between_Feature_selected"] = all_accuracy[:46]

52


In [25]:
df.to_csv('.../table.csv')

NameError: name 'df' is not defined

In [22]:
"""
Within 
"""
all_accuracy = []
# train a model 
# return y_test, y_pred 
avg_accuracy = []
for d in range(x.shape[0]):
    # training set and test set
    accuracy =[]
    print("within # person:",d)
    for i in range(x[d].shape[0]):
        x_train,x_test = get_x_train_test(x[d],i)
        y_train,y_test = get_y_train_test(y[d],i)
        y_test,y_pred = model_train(x_train,x_test,y_train,y_test,model)
        accuracy.append(accuracy_score(y_test,y_pred))
    avg_accuracy.append(sum(accuracy)/len(accuracy))
    all_accuracy.append(accuracy)

within # person: 0
within # person: 1
within # person: 2
within # person: 3
within # person: 4
within # person: 5
within # person: 6
within # person: 7
within # person: 8
within # person: 9
within # person: 10
within # person: 11
within # person: 12
within # person: 13
within # person: 14
within # person: 15
within # person: 16
within # person: 17
within # person: 18
within # person: 19
within # person: 20
within # person: 21
within # person: 22
within # person: 23
within # person: 24
within # person: 25
within # person: 26
within # person: 27
within # person: 28
within # person: 29
within # person: 30
within # person: 31
within # person: 32
within # person: 33
within # person: 34
within # person: 35
within # person: 36
within # person: 37
within # person: 38
within # person: 39
within # person: 40
within # person: 41


In [23]:
print(avg_accuracy)

[0.7222222222222222, 0.6666666666666666, 0.6666666666666666, 0.5555555555555556, 0.6111111111111112, 0.75, 0.8055555555555556, 0.6111111111111112, 0.6111111111111112, 0.5, 0.7333333333333333, 0.9, 0.6666666666666666, 0.4666666666666667, 0.5833333333333334, 0.75, 0.5555555555555556, 0.5277777777777778, 0.7916666666666666, 0.6388888888888888, 0.6944444444444444, 0.5555555555555556, 0.5, 0.6388888888888888, 0.7222222222222222, 0.6944444444444444, 0.6944444444444444, 0.5277777777777778, 0.5555555555555556, 0.7222222222222222, 0.9444444444444444, 0.8055555555555556, 0.75, 0.6666666666666666, 0.6944444444444444, 0.7777777777777778, 0.6666666666666666, 0.6111111111111112, 0.8055555555555556, 0.6666666666666666, 0.4666666666666667, 0.4666666666666667]


In [38]:
for i in range(len(avg_accuracy)):
    avg_accuracy[i] = round(avg_accuracy[i],3)

In [29]:
df = pd.read_csv('../table.csv')
for i in range(10):
    avg_accuracy.append(-1)
df["within_Feature_Less_RF"] = avg_accuracy[:52]
df.to_csv('../table.csv')

In [40]:
df.to_csv('table.csv')

In [92]:
for _ in range(22):
    avg_accuracy.append(-1)

In [122]:
for i in range(4):
    avg_accuracy.append(-1)
df["within_Feature_asymmtry"] = avg_accuracy[:46]

In [123]:
for i in range(4):
    avg_accuracy.append(-1)
df["within_Feature_asymmtry"] = avg_accuracy[:46]

In [97]:
sum(accs[:42])/len(accs[:42])

0.6708571428571426

In [96]:
accs

[0.722,
 0.611,
 0.722,
 0.611,
 0.722,
 0.722,
 0.861,
 0.639,
 0.667,
 0.639,
 0.8,
 0.5,
 0.583,
 0.667,
 0.611,
 0.639,
 0.694,
 0.5,
 0.583,
 0.583,
 0.722,
 0.806,
 0.639,
 0.694,
 0.667,
 0.611,
 0.528,
 0.778,
 0.778,
 0.639,
 0.778,
 0.694,
 0.722,
 0.5,
 0.778,
 0.833,
 0.7,
 0.7,
 0.533,
 0.639,
 0.667,
 0.694,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]